<a href="https://colab.research.google.com/github/aml7hawaiiedu/CCAPLandCoverProject/blob/main/CCAP_UNET_Fall2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install git+https://github.com/facebookresearch/segment-anything.git

In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

In [ ]:
# import urllib.request

# url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"  # Replace with the URL of the file you want to download
# save_path = '/content/sam_vit_h_4b8939.pth'  # Replace with the desired path and file name to save

# urllib.request.urlretrieve(url, save_path)

In [ ]:
# from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

In [3]:
import rasterio
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
import os
import glob
import os
import gdown
import zipfile
from skimage.transform import resize
from tensorflow.keras import layers
import random
from keras.utils import to_categorical
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
import cv2
from scipy.ndimage import generic_filter
from keras.models import load_model
from scipy.stats import mode
import glob
import os
import cv2
import numpy as np
import rasterio
from keras.utils import Sequence, to_categorical
import rasterio.plot
import glob
# Additional code can be added here if needed

In [4]:
os.mkdir('/content/image_chips')

FileExistsError: ignored

In [5]:
import glob
import os

# Get all .zip files
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')

destination_path = '/content/image_chips/'

for zip_file in zip_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(zip_file)[:-4]

    # Create a new directory for this zip file
    unzip_dir = os.path.join(destination_path, base_name)
    os.makedirs(unzip_dir, exist_ok=True)

    # Unzip into the specific directory
    !unzip -q "$zip_file" -d "$unzip_dir"


warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Niihau_subset.zip]:  zipfile is empty
warning [/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/Lanai_subset.zip]:  zipfile is empty


In [7]:
import glob
import os
import pandas as pd

# Get all .csv files
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')


csv_list = []
for csv_file in csv_files:
    # Get the base name of the zip file (without .zip extension)
    base_name = os.path.basename(csv_file)[:-4]

    csv_loc = os.path.join(destination_path, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)


index_csv = pd.concat(csv_list, ignore_index=True)


In [13]:
print(index_csv.groupby('label')['percent'].sum())

print(index_csv.groupby('label')['percent'].count())

index_csv.groupby('label')['percent'].sum()/index_csv.groupby('label')['percent'].count()

label
13    339.165585
14     80.866539
15     26.158833
16      5.413750
17      3.640327
18      4.082653
Name: percent, dtype: float64
label
13    1431
14    1117
15    1285
16     199
17     205
18     170
Name: percent, dtype: int64


label
13    0.237013
14    0.072396
15    0.020357
16    0.027205
17    0.017758
18    0.024016
Name: percent, dtype: float64

In [ ]:
'/content/image_chips/Maui10_subset/Images/10241_10753.tif'

In [14]:
index_csv

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,label,percent,tif_name,subset
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,13,0.000919,8705_513.tif,Kauai10_subset
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,13,0.033161,9217_513.tif,Kauai10_subset
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,13,0.000130,13313_513.tif,Kauai10_subset
3,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,14,0.000137,13313_513.tif,Kauai10_subset
4,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,15,0.003502,13313_513.tif,Kauai10_subset
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4402,1537,1025,1025,513,26,703456.408765,704685.207789,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai_subset
4403,1537,1025,1025,513,26,703456.420411,704685.233874,2.313662e+06,2.314891e+06,15,0.000828,1025_513.tif,Lanai11_subset
4404,7169,5121,6657,4609,221,716973.164648,718201.965448,2.303832e+06,2.305061e+06,15,0.001396,6657_4609.tif,Lanai11_subset
4405,7681,8705,7169,8193,383,718201.993245,719430.775154,2.295230e+06,2.296459e+06,15,0.001637,7169_8193.tif,Lanai11_subset
